In [1]:
# Initiating Required Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import date
import statistics
import scipy.linalg as la
from collections import Counter
from pytrie import StringTrie
import scipy.linalg as la
from factor_analyzer import FactorAnalyzer
from sklearn import preprocessing
from sklearn.metrics import silhouette_samples, silhouette_score

**Functions**

In [2]:
# Function: to find the Strings with first few letters e.g. 'ASE', 'Z_ASE'
def prefixSearch(arr,prefix): 
    trie=StringTrie() 
    for key in arr: 
        trie[key] = key 
    return trie.values(prefix) 

In [3]:
# Function to Create Scree Plot and find optimal No. of Clusters
def scree(df, col_name, n_clust = 10):
    sse = {}
    for k in range(1, n_clust):
        kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df.filter(prefixSearch(df,col_name)))
        df["clusters"] = kmeans.labels_
        #print(data["clusters"])
        sse[k] = kmeans.inertia_ # Inertia: Sum of squared distances of samples to their closest cluster center.
    plt.figure()
    plt.plot(list(sse.keys()), list(sse.values()))
    plt.xlabel("Number of cluster")
    plt.ylabel("Within Cluster SSE")
    plt.title("Scree Plot")
    plt.show()

In [4]:
# Function: Defining the K-Means Clusters with option to provide a file with initial Centroid Means
def kmean_clusters(dataset, col_name, Nclust = 5, init_means = 0, init_cent_means_df = np.empty([2,2])):
    df1 = dataset
    if init_means == 0:
        # Cluster Analysis
        clusters = KMeans(n_clusters=Nclust, n_init = 50).fit(df1.filter(prefixSearch(df1,col_name)))
        # Cluster Centroid
        centers = pd.DataFrame(clusters.cluster_centers_)
        centers.columns = df1.filter(prefixSearch(df1,col_name)).columns
        centers.to_csv("Cluster_Centers"+ str(Nclust) +str(date.today()) + ".csv")
        # Cluster Label join with df
        df1["Clusters_" + col_name] = clusters.labels_
        #df1.rename(columns={"Clusters": col_name + "_Clusters"})
        return df1
    else:
        # Cleaning the Initial Central Means
        cent_init = init_cent_means_df
        cent_init = cent_init.drop("Unnamed: 0", axis = 1)
        # Cluster Analysis
        clusters = KMeans(n_clusters=Nclust, init = cent_init, n_init = 1).fit(df1.filter(prefixSearch(df1,col_name)))
        # Cluster Centroid
        centers = pd.DataFrame(clusters.cluster_centers_)
        centers.columns = df1.filter(prefixSearch(df1,col_name)).columns
        centers.to_csv("Cluster_Centers_usedInit_"+ str(Nclust) + str(date.today()) + ".csv")
        # Cluster Label join with df
        df1["Clusters_" + col_name] = clusters.labels_
        #df1.rename(columns={"Clusters": col_name + "_Clusters"})
        return df1

In [5]:
# Function: Run Multiple Clusters from 2 to n
def multi_kMeans(df,col_name,n):
    for i in range(2,n+1):
        df_clust = df
        #km = KMeans(n_clusters=i, random_state=0, n_init = 50).fit(df_clust.filter(prefixSearch(df_clust,col_name)))
        km = KMeans(n_clusters=i, n_init = 50).fit(df_clust.filter(prefixSearch(df_clust,col_name)))
        kname=col_name + "_segment_"+str(i)
        df_clust[kname] = km.labels_
    return df_clust

In [6]:
# Function: Cross Tab1
def Xtab1(df, seg_col, col_name,aggfunc='mean'):
    results=pd.pivot_table(df.filter(prefixSearch(df.loc[:,x],col_name)), index=df[seg_col],
                       aggfunc='mean',margins=True,margins_name="Total_mean")
    Xtab = results.transpose()
    return Xtab

In [7]:
# Function to Calculate RI Index
def RiIndex(xtab,df,col_names):
    N = len(prefixSearch(df.loc[:,x],col_names))
    ri_ix = xtab.apply(lambda x : x * N)
    return ri_ix

In [8]:
# Function: Cross Tab to show segment X Attribute
def Xtab_attribute(df, attrib, seg_col):
    Result2 = pd.crosstab(df[attrib] , df[seg_col]).apply(lambda r: (r/r.sum())*100, axis=0)
    return Result2

In [9]:
# Function to Calculate Cleaned (Z_ASE) scores from Raw ASE Scores
def z_ASE_calc(df,col_name = 'ASE'):
    outlier_ratio = 600 / df.filter(prefixSearch(df,col_name)).shape[1]
    for i in prefixSearch(df,col_name):
        df.loc[:,"Z_"+i] = df.loc[:,i].apply(lambda x: outlier_ratio if x > outlier_ratio else x)
    df["r_sum"] = df.filter(prefixSearch(df,"Z_ASE")).sum(axis=1)
    for i in prefixSearch(df,"Z_"+col_name):
        df.loc[:,i] = df.loc[:,i]/df["r_sum"] * 100    
    return df

In [10]:
# Function to calculate Deciles for each column
def calc_declie(df,col_name = 'Z_ASE'):
    outlier_ratio = 600 / df.filter(prefixSearch(df,col_name)).shape[1]
    for i in prefixSearch(df,col_name):
        df.loc[:,"dec_"+i] = pd.qcut(df.loc[:,i],10, labels=False)
    return df

In [11]:
# Function to calculate Normalized Score for each respondent
def row_scale(df,col_name):
    x=pd.DataFrame(preprocessing.scale(df.filter(prefixSearch(df,col_name)), axis=1))
    x.columns="Std_"+df.filter(prefixSearch(df,col_name)).columns
    df1 = pd.concat([df, x], axis=1)
    return df1

In [12]:
# Function to calculate No of Factors based on Eigen Values
def eigenvalues(data,col_name):
    eigvals=np.array(data.filter(prefixSearch(df,col_name)))
    eigvals=eigvals.T
    corrmat=np.corrcoef(eigvals)
    eigenvalues, eigenvectors = np.linalg.eig(corrmat)
    Count=Counter(eigenvalues>1)[1]
    return Count,eigenvectors

In [13]:
# Function to calculate Factor Scores
def FacterCluster(df,col_name,rotation = 'varimax'):
    fa = FactorAnalyzer(n_factors=eigenvalues(df,col_name)[0],rotation=rotation,method='ml',use_smc=True)
    df1=pd.DataFrame.from_records(fa.fit_transform(df.filter(regex=col_name)))
    df1=pd.DataFrame(df1)
    df1=df1.add_prefix('Factor' + col_name)
    df2 = pd.concat([df, df1], axis=1)
    return df2

In [14]:
# Function to Drom columns that are not required posr factor analysis
def Drop_col(factor_data):
    df2=pd.DataFrame(factor_data)
    df2.reset_index(inplace=True)
    df2=pd.DataFrame(df2)
    df2=df2.drop(['index'],axis=1)
    return df2

In [15]:
# Function to Calculate Pseudo-F score and identify the best segmentation solution
def get_pseudoF(df, seg, col_name = 'Z_ASE'):
    #seg = "dec_Z_ASE_segment_6"
    Nj = pd.DataFrame(df[seg].value_counts())
    xj = df.groupby(seg).mean().filter(prefixSearch(df.loc[:,x],col_name))
    xdbar = df.filter(prefixSearch(df.loc[:,x],col_name)).mean()

    wsse = 0
    for j in Nj.index:
        xi = df[df[seg] == j].filter(prefixSearch(df.loc[:,x],col_name))
        n = ((xi - xj[xj.index == j].to_numpy())**2).sum().sum()
        wsse = wsse + n

    asse_a = ((xj-xdbar)**2)
    asse = 0
    for j in Nj.index:
        m = (asse_a[asse_a.index == j] * Nj[Nj.index==j].to_numpy()).sum().sum()
        asse = m + asse

    S = df[seg].nunique()
    N = df.shape[0]

    pseudo_f = (asse / (S-1)) /(wsse / (N - S))
    return(pseudo_f)

In [16]:
# Function to calculate Silhouette Avg
def get_silhouette(df, seg, col_name = 'Z_ASE'):
    silhouette_avg = silhouette_score(df.filter(prefixSearch(df,col_name)).values, df[seg])
    return silhouette_avg

**Data**

In [17]:
# Read Required Data Frame
# imoprt pandas as pd
df = pd.read_excel("Raw_Input_Segmentation.xlsx")
df = z_ASE_calc(df, "ASE")
df = calc_declie(df,col_name = 'Z_ASE')
df = FacterCluster(df,'dec_Z_ASE')
df = row_scale(df, 'Z_ASE')
df = FacterCluster(df,'Z_ASE')
x = df.columns

C:\Users\Sharath Chandra\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\Sharath Chandra\Anaconda3\lib\site-packages\factor_analyzer\factor_analyzer.py:418: RuntimeWarning: invalid value encountered in log
  n_factors + corr_mtx.shape[0])
C:\Users\Sharath Chandra\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:390: RuntimeWarning: invalid value encountered in less
  if np.any((x0 < lb) | (x0 > ub)):
C:\Users\Sharath Chandra\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:390: RuntimeWarning: invalid value encountered in greater
  if np.any((x0 < lb) | (x0 > ub)):
C:\Users\Sharath Chandra\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:405: RuntimeWarning: invalid value encountered in greater_equal
  sign_x0 = (x0 >= 0).astype(float) 

In [18]:
# User input no of clusters - create only 1 solution on N clusters
user_iN = 5
df = kmean_clusters(df, 'Z_ASE', Nclust = user_iN)
# Analysis Z_ASE + Clusters
n = 6
df = multi_kMeans(df, 'Z_ASE', n = n)
# Analysis Z_ASE + Factor + Cluster
df = multi_kMeans(df, col_name = 'Factor', n = n)
# Analysis Case Wise Standardized Cleaned ASE Score + Cluster
df = multi_kMeans(df, col_name = 'Std_Z_ASE', n = n)
# Cleaned ASE Score + Decile + Factor + Cluster
df = multi_kMeans(df, col_name = 'Factordec_Z_ASE', n = n)

In [19]:
segment_cols = [col for col in df.columns if 'segment' in col]
cols = ['SegmentSolution','no_of_segments' ,'PseoduFScore', 'silhoutte_avg']
f_df = pd.DataFrame(columns = cols)

for i in segment_cols:
    f_df = f_df.append({'SegmentSolution': str(i),
                        'no_of_segments':i[-1] ,
                        'PseoduFScore': str(get_pseudoF(df,i, d.loc[d['find'] == i.split('_',1)[0]].iloc[0,0])),
                        'silhoutte_avg' : str(get_silhouette(df, i, d.loc[d['find'] == i.split('_',1)[0]].iloc[0,0]))},ignore_index=True)
f_df.sort_values(by=['no_of_segments'])

NameError: name 'd' is not defined

**Create Cross Tab**

In [22]:
prop = df['Factordec_Z_ASE_segment_6'].value_counts(normalize = True)
prop * 100

2    35.439056
3    20.812582
0    14.023591
4    10.249017
1    10.170380
5     9.305374
Name: Factordec_Z_ASE_segment_6, dtype: float64

In [23]:
xt1 = Xtab1(df, "Factordec_Z_ASE_segment_4", col_name = "Z_ASE")

In [24]:
RiIndex(xtab = xt1, df = df, col_names = "Z_ASE")

Factordec_Z_ASE_segment_4,0,1,2,3,Total_mean
Z_ASE1,103.541093,101.315853,94.565975,97.683704,99.834682
Z_ASE10,65.496887,91.221924,66.769817,60.475102,79.120734
Z_ASE11,55.034039,95.912738,93.249027,85.777957,89.665814
Z_ASE12,86.196124,97.624771,94.404161,78.804714,92.266706
Z_ASE13,89.995743,106.691749,98.318222,140.373867,110.841540
Z_ASE14,32.127735,91.138472,31.634189,105.835208,80.303650
Z_ASE15,28.891085,94.114157,23.401147,138.536901,87.199260
Z_ASE16,74.395781,105.106081,72.720681,154.483392,107.789326
Z_ASE17,301.144393,108.383071,124.202889,60.829498,118.828585
Z_ASE18,63.768799,96.316887,96.327503,64.963380,86.887458


In [25]:
Xtab_attribute(df = df, attrib="Product_Usage_Feb26" , seg_col = "Z_ASE_segment_5")

Z_ASE_segment_5,0,1,2,3,4
Product_Usage_Feb26,,,,,
1,48.975324,50.000000,77.486911,63.888889,43.609023
2,7.402760,8.431373,2.617801,7.098765,16.791980
3,4.684233,6.862745,3.141361,3.086420,8.270677
4,14.219992,17.450980,9.424084,16.666667,16.541353
5,24.717691,17.254902,7.329843,9.259259,14.786967
